In [4]:
import numpy as np

train_data = np.load("C:/Users/sdat7/home/2021/기계학습/[2021 Spring Class] Ajou ML by JBR/train_data.npy", allow_pickle=True)
train_label = np.load("C:/Users/sdat7/home/2021/기계학습/[2021 Spring Class] Ajou ML by JBR/train_label.npy", allow_pickle=True)

train_data.shape

(2682,)